In [2]:
target_list = ('crosswalk', 'roadway', 'sidewalk')

def coco_form():
    coco_format = dict(
        info=dict(
            year=2021, version="1.0", description="Pedestrian", contributor="AI Hub", url="https://aihub.or.kr/aidata/136", date_created='2021-11-26 02:39:00'
        ),
        licenses=[
            dict(id=0, name='', url='')
        ],
        categories=[
            dict(id=0, name='background', supercategory=None),
            dict(id=1, name='crosswalk', supercategory=None),
        ],
        images=[
        ],
        annotations=[
        ]
    )

    # with open('coco_format.json','w') as f:
    #     json.dump(coco_format, f, indent=4)

    return coco_format

In [3]:
coco_form()

{'info': {'year': 2021,
  'version': '1.0',
  'description': 'Pedestrian',
  'contributor': 'AI Hub',
  'url': 'https://aihub.or.kr/aidata/136',
  'date_created': '2021-11-26 02:39:00'},
 'licenses': [{'id': 0, 'name': '', 'url': ''}],
 'categories': [{'id': 0, 'name': 'background', 'supercategory': None},
  {'id': 1, 'name': 'crosswalk', 'supercategory': None}],
 'images': [],
 'annotations': []}

## Dataframe to COCO_Format
### 작업 순서
1. 중복없이 파일 이름만 빼온다
2. 파일 이름의 길이만큼 for 문을 돌리는데
- 각 파일 이름끼리 group 을 묶어서 좌표를 다 가져오고
- 그 정보를 cocoformat 형태에 넣는다.
- area 계산도 해서 넣어야 한다.

In [4]:
import pandas as pd

In [5]:
# file in list
# https://jaeyung1001.tistory.com/44
from ast import literal_eval
test_xml = pd.read_csv('/opt/ml/team_github/utils/crosswalk_3_seg.csv')
test_xml

,root_folder,sub_folder,xml_name,file_name,occluded,classes,polygon,subclass,final_class
0,Surface_1,Surface_004,18_SM0915_04.xml,MP_SEL_SUR_000433.jpg,0,sidewalk,"0.00,464.60;152.60,400.00;159.30,413.60;154.64...",asphalt,sidewalk
1,Surface_1,Surface_006,20_SM0915_06.xml,MP_SEL_SUR_000671.jpg,0,sidewalk,"0.00,708.33;273.58,466.22;393.32,401.74;418.32...",asphalt,sidewalk
2,Surface_1,Surface_006,20_SM0915_06.xml,MP_SEL_SUR_000671.jpg,0,sidewalk,"1691.22,97.91;1682.30,137.90;1735.60,116.60",asphalt,sidewalk
3,Surface_1,Surface_010,24_SM0915_10.xml,MP_SEL_SUR_001086.jpg,0,sidewalk,"994.40,1080.00;1356.90,772.80;1678.60,525.70;1...",asphalt,sidewalk
4,Surface_1,Surface_016,30_SM0915_16.xml,MP_SEL_SUR_001886.jpg,0,sidewalk,"326.64,1080.00;385.24,876.03;447.10,698.60;611...",asphalt,sidewalk
...,...,...,...,...,...,...,...,...,...
12542,Surface_5,Surface_700,705_SM1104_03.xml,MP_SEL_SUR_036078.jpg,0,sidewalk,"302.40,231.11;0.00,201.81;0.00,553.41",urethane,sidewalk
12543,Surface_4,Surface_615,758_SM1030_IMT10.xml,MP_SEL_SUR_038879.jpg,0,sidewalk,"810.70,0.00;550.30,250.60;337.40,624.60;25.50,...",urethane,sidewalk
12544,Surface_4,Surface_615,758_SM1030_IMT10.xml,MP_SEL_SUR_038879.jpg,0,sidewalk,"1355.28,1080.00;1131.00,381.40;1060.59,344.62;...",urethane,sidewalk
12545,Surface_5,Surface_635,772_SM1031_IMT04.xml,MP_SEL_SUR_039548.jpg,0,sidewalk,"183.11,163.24;587.50,125.90;604.59,125.27;607....",urethane,sidewalk


In [6]:
target_list = ('crosswalk', 'roadway', 'sidewalk')

mapped_classes = test_xml.copy()
mapped_classes['final_class'] = test_xml['final_class'].replace(['crosswalk', 'roadway', 'sidewalk'], [1, 2, 3])

mapped_classes

,root_folder,sub_folder,xml_name,file_name,occluded,classes,polygon,subclass,final_class
0,Surface_1,Surface_004,18_SM0915_04.xml,MP_SEL_SUR_000433.jpg,0,sidewalk,"0.00,464.60;152.60,400.00;159.30,413.60;154.64...",asphalt,3
1,Surface_1,Surface_006,20_SM0915_06.xml,MP_SEL_SUR_000671.jpg,0,sidewalk,"0.00,708.33;273.58,466.22;393.32,401.74;418.32...",asphalt,3
2,Surface_1,Surface_006,20_SM0915_06.xml,MP_SEL_SUR_000671.jpg,0,sidewalk,"1691.22,97.91;1682.30,137.90;1735.60,116.60",asphalt,3
3,Surface_1,Surface_010,24_SM0915_10.xml,MP_SEL_SUR_001086.jpg,0,sidewalk,"994.40,1080.00;1356.90,772.80;1678.60,525.70;1...",asphalt,3
4,Surface_1,Surface_016,30_SM0915_16.xml,MP_SEL_SUR_001886.jpg,0,sidewalk,"326.64,1080.00;385.24,876.03;447.10,698.60;611...",asphalt,3
...,...,...,...,...,...,...,...,...,...
12542,Surface_5,Surface_700,705_SM1104_03.xml,MP_SEL_SUR_036078.jpg,0,sidewalk,"302.40,231.11;0.00,201.81;0.00,553.41",urethane,3
12543,Surface_4,Surface_615,758_SM1030_IMT10.xml,MP_SEL_SUR_038879.jpg,0,sidewalk,"810.70,0.00;550.30,250.60;337.40,624.60;25.50,...",urethane,3
12544,Surface_4,Surface_615,758_SM1030_IMT10.xml,MP_SEL_SUR_038879.jpg,0,sidewalk,"1355.28,1080.00;1131.00,381.40;1060.59,344.62;...",urethane,3
12545,Surface_5,Surface_635,772_SM1031_IMT04.xml,MP_SEL_SUR_039548.jpg,0,sidewalk,"183.11,163.24;587.50,125.90;604.59,125.27;607....",urethane,3


In [7]:
# define fields
# images
LICENCE = 0
URL = None
DATA_CAPTURED = None
WIDTH = 1920
HEIGHT = 1080
BBOX = []
AREA = 0
ISCROWD = 0

file_list = mapped_classes["file_name"].tolist()
set_file_list = set(file_list)
print(f'total #bbox : {len(file_list)},\timages: {len(set_file_list)}')

total #bbox : 12547,	images: 1895


In [8]:
set_file_list = sorted(set_file_list)
set_file_list

['MP_SEL_SUR_000033.jpg',
 'MP_SEL_SUR_000066.jpg',
 'MP_SEL_SUR_000084.jpg',
 'MP_SEL_SUR_000110.jpg',
 'MP_SEL_SUR_000119.jpg',
 'MP_SEL_SUR_000144.jpg',
 'MP_SEL_SUR_000161.jpg',
 'MP_SEL_SUR_000312.jpg',
 'MP_SEL_SUR_000390.jpg',
 'MP_SEL_SUR_000396.jpg',
 'MP_SEL_SUR_000418.jpg',
 'MP_SEL_SUR_000433.jpg',
 'MP_SEL_SUR_000489.jpg',
 'MP_SEL_SUR_000493.jpg',
 'MP_SEL_SUR_000494.jpg',
 'MP_SEL_SUR_000519.jpg',
 'MP_SEL_SUR_000523.jpg',
 'MP_SEL_SUR_000531.jpg',
 'MP_SEL_SUR_000567.jpg',
 'MP_SEL_SUR_000569.jpg',
 'MP_SEL_SUR_000570.jpg',
 'MP_SEL_SUR_000571.jpg',
 'MP_SEL_SUR_000574.jpg',
 'MP_SEL_SUR_000611.jpg',
 'MP_SEL_SUR_000615.jpg',
 'MP_SEL_SUR_000621.jpg',
 'MP_SEL_SUR_000627.jpg',
 'MP_SEL_SUR_000635.jpg',
 'MP_SEL_SUR_000643.jpg',
 'MP_SEL_SUR_000652.jpg',
 'MP_SEL_SUR_000654.jpg',
 'MP_SEL_SUR_000670.jpg',
 'MP_SEL_SUR_000671.jpg',
 'MP_SEL_SUR_000701.jpg',
 'MP_SEL_SUR_000727.jpg',
 'MP_SEL_SUR_000745.jpg',
 'MP_SEL_SUR_000748.jpg',
 'MP_SEL_SUR_000795.jpg',
 'MP_SEL_SUR

In [9]:
from tqdm import tqdm
image_id_index = 0
coco = coco_form()
for image in tqdm(set_file_list):
    condition = mapped_classes['file_name'] == image
    coco['images'].append(
        dict(license=LICENCE, url=URL, file_name=image, height=HEIGHT, width=WIDTH, data_captured=DATA_CAPTURED, id=image_id_index)
        )

    box_id_index = 0
    for polygon in mapped_classes[condition].to_dict('records'):
        SEGMENTATION = list()
        
        polygons = polygon['polygon'].split(';')

        for polygon_xy in polygons:
            X, Y = polygon_xy.split(',')
            SEGMENTATION.append(int(float(X)))
            SEGMENTATION.append(int(float(Y)))
        SEGMENTATION.append(SEGMENTATION[0])
        SEGMENTATION.append(SEGMENTATION[1])

        coco['annotations'].append(
            dict(id=box_id_index, image_id=image_id_index, category_id=polygon['final_class'], bbox=BBOX, 
                area=AREA, segmentation=[SEGMENTATION], iscrowd=ISCROWD)
            )
        box_id_index+=1
    image_id_index+=1

100%|██████████| 1895/1895 [00:03<00:00, 505.07it/s]


In [10]:
# 잘 들어갔는지 검증
print(f'total #bbox : {len(coco["annotations"])},\timages: {len(coco["images"])}')

total #bbox : 12547,	images: 1895


In [11]:
import json
with open('crosswalk_filtering_closed(12_12).json', 'w') as f:
    json.dump(coco,f, indent=4)